# Semi-supervised Pipeline for Micropaleontological Slide Image Analysis - PART I

## Set up

### Library

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from tensorflow import keras
import zipfile

import mahotas as mt

from keras.models import load_model, Sequential
from keras.utils import img_to_array
from keras import optimizers, layers
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Model

from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.metrics import silhouette_score

import skimage
from skimage.filters import threshold_multiotsu

### Functions

In [ ]:
### 
### function used for plotting segmented image after SAM
###
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

### 
### function used to delete all png files in a certain directory
###
def delete_png_files(directory):
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Deleted file: {file_path}")
            count += 1
    print(f"Total files deleted: {count}")

###
### function used to save png images into a zip file
###
def save_png_as_zip(folder_path, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w') as zip:
        for filename in os.listdir(folder_path):
            if filename.endswith('.png'):
                image_path = os.path.join(folder_path, filename)
                zip.write(image_path, filename)

###
### function used to plot randomly sampled images from each cluster
###
def plot_random_samples(df, image_dir, num_samples=3):
    # Initialize an empty dataframe to store the randomly sampled rows
    sampled_df = pd.DataFrame()

    # Iterate over unique cluster labels
    for cluster_label in df['ClusterLabel'].unique():
        # Filter the dataframe to include only rows with the current cluster label
        cluster_df = df[df['ClusterLabel'] == cluster_label]

        # Randomly sample rows from the current cluster
        random_sample = cluster_df.sample(n=num_samples, random_state=42)  # Adjust the number of samples as needed

        # Append the sampled rows to the new dataframe
        sampled_df = pd.concat([sampled_df, random_sample])

    # Group the dataframe by 'ClusterLabel'
    grouped_df = sampled_df.groupby('ClusterLabel')

    # Iterate over the groups and plot the images
    for cluster_label, group in grouped_df:
        # Create a subplot grid for the images in the current cluster
        num_images = len(group)
        num_cols = 5
        num_rows = (num_images - 1) // num_cols + 1
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, num_rows*2))
        axes = axes.flatten()

        # Iterate over the images in the current cluster and plot them
        for i, (_, row) in enumerate(group.iterrows()):
            filename = row['filename']
            i_value = row['i']
            image_path = os.path.join(image_dir, f'ROI_{i_value}_{filename}.png')
            img = plt.imread(image_path)

            axes[i].imshow(img)
            axes[i].axis('off')
            axes[i].set_title(f'Cluster: {cluster_label}')

        # Remove any empty subplots
        for j in range(num_images, num_rows*num_cols):
            fig.delaxes(axes[j])

        # Adjust the spacing between subplots
        plt.tight_layout()

        # Show the plot for the current cluster
        plt.show()

###
### function used to plot all the images in one specific cluster
###
def plot_cluster_images(df, image_dir, cluster_label):
    # Filter the dataframe to include only rows with the specified cluster label
    cluster_df = df[df['ClusterLabel'] == cluster_label]

    # Get the number of images in the cluster
    num_images = len(cluster_df)

    # Create a subplot grid for the images in the cluster
    num_cols = 5
    num_rows = (num_images - 1) // num_cols + 1
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, num_rows*2))
    axes = axes.flatten()

    # Iterate over the images in the cluster and plot them
    for i, (_, row) in enumerate(cluster_df.iterrows()):
        filename = row['filename']
        i_value = row['i']
        image_path = os.path.join(image_dir, f'ROI_{i_value}_{filename}.png')
        img = plt.imread(image_path)

        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(f'Cluster: {cluster_label}')

    # Remove any empty subplots
    for j in range(num_images, num_rows*num_cols):
        fig.delaxes(axes[j])

    # Adjust the spacing between subplots
    plt.tight_layout()

    # Show the plot for the specified cluster
    plt.show()

###
### function used to save all the images that are not in specified clusters
###
def save_non_cluster_images(df, image_dir, excluded_cluster_labels, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for cluster_label in df['ClusterLabel'].unique():
        # Skip the cluster if it is in the excluded_cluster_labels
        if cluster_label in excluded_cluster_labels:
            continue

        # Filter the dataframe to include only rows with the specified cluster label
        cluster_df = df[df['ClusterLabel'] == cluster_label]

        # Iterate over the images in the cluster and save them to the output directory
        for _, row in cluster_df.iterrows():
            filename = row['filename']
            i_value = row['i']
            image_path = os.path.join(image_dir, f'ROI_{i_value}_{filename}.png')
            output_path = os.path.join(output_dir, f'ROI_{i_value}_{filename}.png')
            shutil.copyfile(image_path, output_path)

        print(f"Saved {len(cluster_df)} images from Cluster {cluster_label} to {output_dir}.")


### 
### function used to save all the images in one specific cluster to a new folder
###
def save_cluster_images(df, image_dir, cluster_labels, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for cluster_label in cluster_labels:

    # Filter the dataframe to include only rows with the specified cluster label
        cluster_df = df[df['ClusterLabel'] == cluster_label]

    # Iterate over the images in the cluster and save them to the output directory
        for _, row in cluster_df.iterrows():
            filename = row['filename']
            i_value = row['i']
            image_path = os.path.join(image_dir, f'ROI_{i_value}_{filename}.png')
            output_path = os.path.join(output_dir, f'ROI_{i_value}_{filename}.png')
            shutil.copyfile(image_path, output_path)

        print(f"Saved {len(cluster_df)} images from Cluster {cluster_label} to {output_dir}.")

###
### function used to save all the images in one specific cluster to a new folder
###
def save_predicted_images(df, image_dir, predicted_labels, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for predicted_label in predicted_labels:

    # Filter the dataframe to include only rows with the specified cluster label
        predicted_df = df[df['predicted_label'] == predicted_label]

    # Iterate over the images in the cluster and save them to the output directory
        for _, row in predicted_df.iterrows():
            filename = row['filename']
            # i_value = row['i']
            image_path = os.path.join(image_dir, f'{filename}')
            output_path = os.path.join(output_dir, f'{filename}')
            shutil.copyfile(image_path, output_path)

        print(f"Saved {len(predicted_df)} images from {predicted_label} to {output_dir}.")

### SAM set up

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda" # cuda default

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_batch = 64, # default
    points_per_side=32, # default
    pred_iou_thresh=0.95, #default of 0.86
    stability_score_thresh=0.92, # default
    # crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=1000,
)

## Image Segmentation by SAM & Feature Extraction

### Example

In [ ]:
sample = cv2.imread('/home/xzhu/517/New folder/SLT49-2ox 14 3 103 0.tif')
sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,20))
plt.axis('off')
plt.imshow(sample)

In [ ]:
segmented_sample = mask_generator.generate(sample)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(sample)
show_anns(segmented_sample)
plt.axis('off')
plt.show() 

### Multi-Otsu Thresholding for Comparison

In [ ]:
# The input image.
sample = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)

# Applying multi-Otsu threshold for the default value, generating
# three classes.
thresholds = threshold_multiotsu(sample)

# Using the threshold values, we generate the three regions.
regions = np.digitize(sample, bins=thresholds)

# Plotting the Multi Otsu result.
plt.imshow(regions, cmap='jet')
#plt.title('Multi-Otsu result')
plt.axis('off')

plt.show()

### Feature Extraction

In [ ]:
folder_path = '/home/xzhu/517/Toarcian AI project inc ox'
delete_png_files(folder_path)

In [ ]:
folder_path = '/home/xzhu/517/New folder'
#delete_png_files(folder_path)

In [ ]:
# Create blank data frames to store the ouputs
df1 = pd.DataFrame(columns = ['filename'])
df2 = pd.DataFrame(columns = ['masks_length'])
df3 = pd.DataFrame(columns = ['i'])
df4 = pd.DataFrame(columns = ['stability_score'])
# Calculating color based features - mean, std-dev of the RGB channels
df5 = pd.DataFrame(columns = ['avg_blue']) #blue, green, red
df6 = pd.DataFrame(columns = ['avg_green'])
df7 = pd.DataFrame(columns = ['avg_red']) 
df8 = pd.DataFrame(columns = ['sd_blue'])
df9 = pd.DataFrame(columns = ['sd_green'])
df10 = pd.DataFrame(columns = ['sd_red'])
# Shape based features calculated - Aspect ratio, rectangularity, circularity etc.
df11 = pd.DataFrame(columns = ['size'])
df12 = pd.DataFrame(columns = ['aspect_ratio'])
df13 = pd.DataFrame(columns = ['rectangularity'])
df14 = pd.DataFrame(columns = ['circularity'])
df15 = pd.DataFrame(columns = ['convexity'])
# Using Haralick moments - calculating texture based features such as contrast, correlation, entropy
df16 = pd.DataFrame(columns = ['contrast'])
df17 = pd.DataFrame(columns = ['correlation'])
df18 = pd.DataFrame(columns = ['entropy'])
df19 = pd.DataFrame(columns = ['inverse_difference_moments'])
# for EF analysis
df20 = pd.DataFrame(columns = ['perimeter'])
#df20 = pd.DataFrame(columns = ['contour_coordinates']) 


# Create blank lists for the for loop
filename_lst = []
masks_lst = []
i_lst = []
stability_score_lst = []
# Color
avg_blue_lst = []
avg_green_lst = []
avg_red_lst = []
sd_blue_lst = []
sd_green_lst = []
sd_red_lst = []
# Shape
size_lst = []
aspect_ratio_lst = []
rectangularity_lst = []
circularity_lst = []
convexity_lst = []
# Texture
contrast_lst = []
correlation_lst = []
entropy_lst =[]
idm_lst = []
# Contour
perimeter_lst = []
#coordinates_lst = []

In [ ]:
# original fossil image folder
folder_path = '/home/xzhu/517/Toarcian AI project inc ox'

for filename in os.listdir(folder_path):
    # Check if the file is a TIF image
    if filename.endswith('.tif'):
        # Read the image using OpenCV
        image = cv2.imread(os.path.join(folder_path, filename))
        # Generate masks for each image
        masks = mask_generator.generate(image)

        for i in range(len(masks)):
            stability_score = masks[i]['stability_score']
            # Create a mask image
            mask_image = np.zeros_like(masks[i]['segmentation'], dtype = np.uint8)
            mask_image[masks[i]['segmentation']] = 255
            # Obtain segments in each image
            segmentation = masks[i]['segmentation']
            # Update mask image with segments
            mask_image = image * segmentation[:,:,np.newaxis] # resize
            # Combine original image with mask image
            result = cv2.bitwise_and(image, mask_image)
            result[mask_image==0] = 0 # black background
        
            ### for output images
            # Create a mask image
            out_image = np.zeros_like(masks[i]['segmentation'], dtype = np.uint8)
            out_image[masks[i]['segmentation']] = 255
            # Obtain segments in each image
            segmentation = masks[i]['segmentation']
            x, y, w, h = masks[i]['bbox']
    
            # Update mask image with segments
            out_image = image * segmentation[:,:,np.newaxis] # resize
            x, y, w, h = int(x), int(y), int(w), int(h)
            out_image = out_image[y:y+h, x:x+w]
    
            # Save each ROI as png file
            cv2.imwrite(os.path.join(folder_path, 'ROI_{}_{}.png'.format(i, filename)), out_image)
            ### end of output images

            # Calculate the image contrast
            img_grey = cv2.cvtColor(out_image, cv2.COLOR_BGR2GRAY)
            contrast = img_grey.std()

            textures = mt.features.haralick(img_grey)
            ht_mean = textures.mean(axis=0)
            correlation = ht_mean[2]
            idm = ht_mean[4] #inverse difference moments
            entropy = ht_mean[8]

            # Calculate the average RGB
            avg_color_per_row = np.average(result, axis = 0)
            avg_color = np.average(avg_color_per_row, axis = 0) #blue, green, red

            # Calculate the standard deviation RGB
            sd_color_per_row = np.std(result, axis = 0)
            sd_color = np.std(sd_color_per_row, axis = 0)

            # Get the mask size
            mask_size = masks[i]['area']
            aspect_ratio = float(w)/h
            rectangularity = mask_size / (w * h)
            
            # Append results to the blank list
            filename_lst.append(filename)
            masks_lst.append(len(masks))
            i_lst.append(i)
            stability_score_lst.append(stability_score)
            avg_blue_lst.append(avg_color[0])
            avg_green_lst.append(avg_color[1])
            avg_red_lst.append(avg_color[2])
            sd_blue_lst.append(sd_color[0])
            sd_green_lst.append(sd_color[1])
            sd_red_lst.append(sd_color[2])
            size_lst.append(mask_size)
            contrast_lst.append(contrast)
            correlation_lst.append(correlation)
            idm_lst.append(idm)
            entropy_lst.append(entropy)

            aspect_ratio_lst.append(aspect_ratio)
            rectangularity_lst.append(rectangularity)
            
            # Create a binary mask image
            co_image = np.zeros_like(masks[i]['segmentation'], dtype = np.uint8)
            co_image[masks[i]['segmentation'] != 0 ] = 255
            # Crop the image with bounding box
            co_image = co_image[y:y+h, x:x+w]

            # Find contours
            contours,_= cv2.findContours(co_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            cnt = contours[0]
            perimeter = cv2.arcLength(cnt,True)
            circularity = ((perimeter)**2)/mask_size
            circularity_lst.append(circularity)
            perimeter_lst.append(perimeter)

            # Convexity
            # Calculate convex hull of the largest contour
            largest_contour = max(contours, key=cv2.contourArea)

            # Create convex hull
            hull = cv2.convexHull(largest_contour, returnPoints=True)
            convex_area = cv2.contourArea(hull)
            convexity = mask_size / convex_area

            convexity_lst.append(convexity)

            # create a blank list for each mask contour coordinates
            #coordinates = []
            #for contour in contours:
            #    # Print the coordinates of each point in the contour
            #    for point in contour:
            #        x, y = point[0]
            #        # save each (x,y) in the blank list
            #        coordinates.append((x,y))
                
            #coordinates_lst.append(coordinates)

In [ ]:
df1['filename'] = filename_lst
df2['masks_length'] = masks_lst
df3['i'] = i_lst
df4['stability_score'] = stability_score_lst
df5['avg_blue'] = avg_blue_lst #blue, green, red
df6['avg_green'] = avg_green_lst
df7['avg_red'] = avg_red_lst
df8['sd_blue'] = sd_blue_lst
df9['sd_green'] = sd_green_lst
df10['sd_red'] = sd_red_lst
df11['size'] = size_lst
df12['aspect_ratio'] = aspect_ratio_lst
df13['rectangularity'] = rectangularity_lst
df14['circularity'] = circularity_lst
df15['convexity'] = convexity_lst
df16['contrast'] = contrast_lst
df17['correlation'] = correlation_lst
df18['entropy'] = entropy_lst
df19['inverse_difference_moments'] = idm_lst
df20['perimeter'] = perimeter_lst
#df20['contour_coordinates'] = coordinates_lst

In [ ]:
df_sum = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, 
                    df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20],axis = 1) # remove contour_coordinates
df_sum

In [ ]:
# save output dataframe as csv file
df_sum.to_csv(os.path.join(folder_path,r'RNS.csv'))
#df_sum.to_csv(os.path.join(folder_path,r'SLT.csv'))

In [ ]:
# save_png_as_zip(folder_path, 'RNS.zip')
# save_png_as_zip(folder_path, 'SLT.zip')

## Image Clustering

In [ ]:
# RNS processed as the example below, SLT was processed in the same way

csv_file = pd.read_csv('/home/xzhu/517/Toarcian AI project inc ox/RNS.csv')

# csv_file = pd.read_csv('/home/xzhu/517/New folder/SLT.csv')

In [ ]:
data = csv_file[['i', 'filename', 'avg_blue', 'avg_green', 'avg_red', 'sd_blue', 'sd_green', 'sd_red', 'size', 'aspect_ratio','rectangularity',
                  'circularity','contrast', 'correlation', 'entropy', 'perimeter']]

# data.head()

In [ ]:
hc_data = data[['avg_blue', 'avg_green', 'avg_red', 'sd_blue', 'sd_green', 'sd_red', 'size', 'aspect_ratio','rectangularity',
                  'circularity','contrast', 'correlation', 'entropy','perimeter']]

In [ ]:
# Scale the hc_data
scaler = StandardScaler()
scaled_hc_data = scaler.fit_transform(hc_data)

### Hybrid Clustering

#### K-means clustering (K = 3)

In [ ]:
k = 3 # Number of clusters
kmeans = KMeans(n_clusters=k)
cluster_labels = kmeans.fit_predict(scaled_hc_data)

# Add cluster labels to the dataframe
data['ClusterLabel'] = cluster_labels # with filename

# for further clustering analysis
scaled_hc_data = pd.DataFrame(scaled_hc_data)
scaled_hc_data['ClusterLabel'] = cluster_labels

In [ ]:
plot_random_samples(data, folder_path, num_samples=5)

In [ ]:
plot_cluster_images(data, folder_path, 2) # plot to overview

In [ ]:
# remove unqualified cluster
print(
    len(scaled_hc_data[scaled_hc_data['ClusterLabel'] == 2])
    )

In [ ]:
# remove masks in dataframe
scaled_hc_data = scaled_hc_data[scaled_hc_data['ClusterLabel'] != 2]
data =  data[data['ClusterLabel'] != 2]

print(hc_data.shape)

# remove clusterlabel column for further clustering
scaled_hc_data = scaled_hc_data.drop(columns=['ClusterLabel'])
print(scaled_hc_data.shape)

data = data.drop(columns=['ClusterLabel'])
print(data.shape)

#### Weighted Hierarchical Clustering

In [ ]:
# create empty array for weights
weights = np.ones(14)

weights[:6] = 0.1/6 # 0.1 for color features
weights[6:10] = 0.6/4 # 0.5 for shape features
weights[10:] = 0.3/4 # 0.4 for texture features

print(weights)

In [ ]:
weighted_data = scaled_hc_data * weights

In [ ]:
# generate the linkage matrix
hc = linkage(weighted_data, 'ward')

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    hc,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

#### Silhouette Analysis

In [ ]:
k_values = range(2, 101)

silhouette_scores = []

for k in k_values:
    clusterer = KMeans(n_clusters=k, n_init=10)
    preds = clusterer.fit_predict(scaled_hc_data)
    centers = clusterer.cluster_centers_

    score = silhouette_score(scaled_hc_data, preds)

    silhouette_scores.append(score)

    print("For n_clusters = {}, silhouette score is {}".format(k, score))


In [ ]:
print(f"Highest Silhouette Score: {max(silhouette_scores)}")
print(f"Corresponding Number of Clusters: {silhouette_scores.index(max(silhouette_scores)) + 2}")

In [ ]:
max_score = max(silhouette_scores)
max_index = silhouette_scores.index(max_score) + 2

plt.figure(figsize=(25, 10))
plt.plot(k_values, silhouette_scores, 'bo-')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for K-means Clustering')

# Plot the highest silhouette score as a marker on the graph
plt.plot(max_index, max_score, 'ro', markersize=10, label='Highest Silhouette Score')

plt.legend()
plt.show()

#### K-means Clustering

In [ ]:
# Dendrogram Truncation
plt.figure(figsize=(20, 7))
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    hc,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=4,  # show only the last p merged clusters
    show_leaf_counts=True,  # numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show()

In [ ]:
k = 100 # Number of clusters
kmeans = KMeans(n_clusters=k)
cluster_labels = kmeans.fit_predict(weighted_data)

# Add cluster labels to the dataframe
data['ClusterLabel'] = cluster_labels
weighted_data['ClusterLabel'] = cluster_labels

In [ ]:
# Dendrogram Truncation
plt.figure(figsize=(20, 7))
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    hc,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=100,  # show only the last p merged clusters
    show_leaf_counts=True,  # numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show()

In [ ]:
plot_random_samples(data, folder_path, num_samples=1)

In [ ]:
plot_cluster_images(data, folder_path, 68)

In [ ]:
data.to_csv(os.path.join(folder_path,r'clustered_RNS.csv'))

In [ ]:
# data.to_csv(os.path.join(folder_path,r'clustered_SLT.csv'))

# Semi-supervised Pipeline for Micropaleontological Slide Image Analysis - PART II

## Semi-supervised Microfossil Image Classification CNN Model

### Data

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
csv = pd.read_csv('/home/xzhu/517/Toarcian AI project inc ox/clustered_RNS.csv')

In [ ]:
image_dir = '/home/xzhu/517/Toarcian AI project inc ox'

fossil_dir = '/home/xzhu/517/cluster_label/fossil'
noise_dir = '/home/xzhu/517/cluster_label/noise'

test ='/home/xzhu/517/test'

In [ ]:
# empty all the folders
delete_png_files(fossil_dir)
delete_png_files(noise_dir)
delete_png_files(test)

In [ ]:
# plot to overview
plot_cluster_images(csv, image_dir, 7)

In [ ]:
# assign images from clusters of interest to fossil/noise folder
fossil_clusters = [29]
save_cluster_images(csv, image_dir, fossil_clusters, fossil_dir)

noise_clusters = [2, 3, 6, 7] # 0, 2, 3, 5, 6, 8
save_cluster_images(csv, image_dir, noise_clusters, noise_dir)

In [ ]:
# rest images were treated as test image
e = [29, 2, 3, 6, 7]
save_non_cluster_images(csv, image_dir, e, test)

In [ ]:
# Manually check images in cluster 29 and remove the following 17 images
filenames_to_remove = ['ROI_13_RNS1-1ox 37 0 101 3.tif.png',
                       'ROI_3_RNS1-1 16 0 99 9.tif.png',
                       'ROI_2_RNS 6-1ox 26 4 106 0.tif.png',
                       'ROI_7_RNS 6-1ox 26 4 106 0.tif.png',
                       'ROI_4_RNS1-1 23 6 100 8.tif.png',
                       'ROI_9_RNS1-1 23 6 100 8.tif.png',
                       'ROI_3_RNS 6-1ox 25 8 106 1.tif.png',
                       'ROI_8_RNS1-1ox 46 5 101 9.tif.png',
                       'ROI_8_RNS1-1 21 7 99 5.tif.png',
                       'ROI_27_RNS1-1ox 42 2 102 3.tif.png',
                       'ROI_2_RNS1-1ox 45 0 100 4.tif.png',
                       'ROI_34_RNS1-1ox 47 1 102 0.tif.png',
                       'ROI_7_RNS1-1ox 46 0 102 1.tif.png',
                       'ROI_0_RNS 6-1ox 30 7 105 6.tif.png',
                       'ROI_7_RNS 6-1ox 28 4 105 7.tif.png',
                       'ROI_9_RNS 6-1ox 28 4 105 7.tif.png',
                       'ROI_38_RNS 6-1ox 28 4 105 7.tif.png']

In [ ]:
for filename in filenames_to_remove:
    file_path = os.path.join(fossil_dir, filename)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed: {file_path}")

In [ ]:
# create training dataset
image_folder = '/home/xzhu/517/cluster_label'
train_ds = tf.keras.utils.image_dataset_from_directory(
  image_folder,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)

In [ ]:
# create a validation split
val_ds = tf.keras.utils.image_dataset_from_directory(
  image_folder,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Model

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
num_classes = len(class_names)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
epochs = 20
history1 = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = []
val_acc = []

loss = []
val_loss = []

In [ ]:
acc1 = history1.history['accuracy']
val_acc1 = history1.history['val_accuracy']

loss1 = history1.history['loss']
val_loss1 = history1.history['val_loss']

acc.extend(acc1)
val_acc.extend(val_acc1)
loss.extend(loss1)
val_loss.extend(val_loss1)


epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### First Training

In [ ]:
csv = pd.read_csv('/home/xzhu/517/Toarcian AI project inc ox/RNS.csv')

In [ ]:
df_filename = pd.DataFrame(columns = ['filename'])
df_label = pd.DataFrame(columns = ['predicted_label'])
df_score = pd.DataFrame(columns = ['score'])

filename_lst = []
predicted_label_lst = []
score_lst = []

In [ ]:
test ='/home/xzhu/517/test'

In [ ]:
# plot the first 50 images one by one
plt.figure(figsize=(5, 6))

# Get the list of image files in the test folder
image_files = [filename for filename in os.listdir(test) if filename.endswith('.png')]

# Iterate over the first 50 images in the test folder and plot them
for filename in image_files[:50]:
    image_path = os.path.join(test, filename)
    img = tf.keras.utils.load_img(image_path, target_size=(224, 224))
        
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
        
    predictions = model.predict(img_array)
    score = tf.nn.sigmoid(predictions[0])

    filename_lst.append(filename)
    predicted_label_lst.append(np.argmax(score))
    score_lst.append(100 * np.max(score))

    plt.imshow(img)
    plt.title("{} most likely belongs to {} with a {:.2f} percent confidence."
              .format(filename, class_names[np.argmax(score)], 100 * np.max(score)))
    plt.axis('off')
    plt.show()

In [ ]:
df_filename['filename'] = filename_lst
df_label['predicted_label'] = predicted_label_lst # 1 = noise, 0 = fossil
df_score['score'] = score_lst

df_sum = pd.concat([csv, df_label, df_score],axis = 1)
df_sum

In [ ]:
model.save("cnn_1.h5")

### Second training

In [ ]:
f = [0]
n = [1]

In [ ]:
save_predicted_images(df_sum, image_dir, f, fossil_dir)

In [ ]:
save_predicted_images(df_sum, image_dir, n, noise_dir)

In [ ]:
# move some identified noise image in fossil folder into noise folder
import shutil

noise_to_move = ['ROI_8_RNS1-1ox 32 5 99 7.tif.png',
                 'ROI_29_RNS 6-1ox 25 1 106 2.tif.png',
                 'ROI_29_RNS1-1 23 6 100 8.tif.png',
                 'ROI_30_RNS1-1ox 45 5 100 0.tif.png',
                 'ROI_33_RNS1-1ox 35 4 101 9.tif.png'
                 ]

for filename in noise_to_move:
    source_path = os.path.join(fossil_dir, filename)
    destination_path = os.path.join(noise_dir, filename)
    shutil.move(source_path, destination_path)
    print(f"Moved: {source_path} to {destination_path}")

In [ ]:
# move some identified fossil image in noise folder into noise folder
fossil_to_move = ['ROI_3_RNS 6-1ox 37 4 105 5.tif.png'
                  ]

for filename in fossil_to_move:
    source_path = os.path.join(noise_dir, filename)
    destination_path = os.path.join(fossil_dir, filename)
    shutil.move(source_path, destination_path)
    print(f"Moved: {source_path} to {destination_path}")

In [ ]:
train_folder = '/home/xzhu/517/cluster_label'
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)

In [ ]:
# create a validation split
val_ds = tf.keras.utils.image_dataset_from_directory(
  image_folder,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
epochs = 20
history2 = model.fit(  #trained_model
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc2 = history2.history['accuracy']
val_acc2 = history2.history['val_accuracy']

loss2 = history2.history['loss']
val_loss2 = history2.history['val_loss']

# append to the previous model training results
acc.extend(acc2)
val_acc.extend(val_acc2)
loss.extend(loss2)
val_loss.extend(val_loss2)


epochs_range = range(40)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
df_filename = pd.DataFrame(columns = ['filename'])
df_label = pd.DataFrame(columns = ['predicted_label'])
df_score = pd.DataFrame(columns = ['score'])

filename_lst = []
predicted_label_lst = []
score_lst = []

In [ ]:
# plot the second 50 images one by one
plt.figure(figsize=(5, 6))

# Get the list of image files in the test folder
image_files = [filename for filename in os.listdir(test) if filename.endswith('.png')]

# Iterate over the first 50 images in the test folder and plot them
for filename in image_files[50:100]:
    image_path = os.path.join(test, filename)
    img = tf.keras.utils.load_img(image_path, target_size=(224, 224))
        
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
        
    predictions = model.predict(img_array)
    score = tf.nn.sigmoid(predictions[0])

    filename_lst.append(filename)
    predicted_label_lst.append(np.argmax(score))
    score_lst.append(100 * np.max(score))

    plt.imshow(img)
    plt.title("{} most likely belongs to {} with a {:.2f} percent confidence."
              .format(filename, class_names[np.argmax(score)], 100 * np.max(score)))
    plt.axis('off')
    plt.show()

In [ ]:
df_filename['filename'] = filename_lst
df_label['predicted_label'] = predicted_label_lst # 1 = noise, 0 = fossil
df_score['score'] = score_lst

df_sum = pd.concat([df_filename, df_label, df_score],axis = 1)
df_sum

In [ ]:
model.save("cnn_2.h5")

### Third Training

In [ ]:
save_predicted_images(df_sum, image_dir, f, fossil_dir)

In [ ]:
save_predicted_images(df_sum, image_dir, n, noise_dir)

In [ ]:
# move some identified noise image in fossil folder into noise folder
noise_to_move = ['ROI_2_RNS 6-1ox 32 0 105 8.tif.png',
                'ROI_8_RNS1-1 25 3 101 6.tif.png',
                'ROI_45_RNS 6-1ox 35 0 105 8.tif.png',
                'ROI_40_RNS 6-1ox 30 0 105 7.tif.png',
                'ROI_2_RNS1-1ox 32 5 99 7.tif.png',
                'ROI_4_RNS 6-1ox 30 7 105 6.tif.png',
                'ROI_22_RNS 6-1ox 30 0 105 7.tif.png',
                'ROI_37_RNS 6-1ox 28 0 105 8.tif.png'
                ]

for filename in noise_to_move:
    source_path = os.path.join(fossil_dir, filename)
    destination_path = os.path.join(noise_dir, filename)
    shutil.move(source_path, destination_path)
    print(f"Moved: {source_path} to {destination_path}")

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)

In [ ]:
# create a validation split
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
epochs = 20
history3 = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc3 = history3.history['accuracy']
val_acc3 = history3.history['val_accuracy']

loss3 = history3.history['loss']
val_loss3 = history3.history['val_loss']

acc.extend(acc3)
val_acc.extend(val_acc3)
loss.extend(loss3)
val_loss.extend(val_loss3)

epochs_range = range(60)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("cnn_3.h5")

### Prediction

In [ ]:
trained_model = tf.keras.models.load_model('cnn_1.h5')

# Semi-supervised Pipeline for Micropaleontological Slide Image Analysis - PART III

In [ ]:
# import csv with predicted labels
inc = pd.read_csv('/home/xzhu/517/predicted_label_inc_84.csv')
new = pd.read_csv('/home/xzhu/517/predicted_label_newfolder_84.csv')

In [ ]:
# 1 = noise, 0 = fossil
inc_f = inc[inc['predicted_label'] == 0]
new_f = new[new['predicted_label'] == 0]

In [ ]:
size_values_inc = inc_f['size']
size_values_new = new_f['size']

In [ ]:
mean_size_inc = inc_f['size'].mean()
mean_size_new = new_f['size'].mean()

In [ ]:
std_size_inc = size_values_inc.std()
std_size_new = size_values_new.std()

In [ ]:
# Data for plotting
dataframes = ['Toarcian pre-CIE', 'Toarcian post-CIE']

# Increase the figure size to make the graph bigger
plt.figure(figsize=(10, 6))

# Create a box plot to show the size values for each DataFrame
boxplot = plt.boxplot([size_values_inc, size_values_new], labels=dataframes, vert=True, patch_artist=True, showfliers=False)

# Customize the appearance of the box plot

# Set box colors
colors = ['lightblue', 'lightgreen']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Set the color of whiskers, caps, and medians
for key in ['whiskers', 'caps', 'medians']:
    for line in boxplot[key]:
        line.set_color('black')

# Set the linewidth of whiskers and caps
for line in boxplot['whiskers'] + boxplot['caps']:
    line.set_linewidth(1.5)

# Set the linewidth and color of medians
for line in boxplot['medians']:
    line.set_linewidth(2)
    line.set_color('red')

# Add gridlines
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.xlabel('Geological Era')
plt.ylabel('Size (in cm2)')
plt.title('Size Comparison between Microfossils from Different Eras')

plt.tight_layout()  # Adjusts spacing to prevent clipping of labels

plt.show()